In [ ]:
# For importing our data
from zipfile import ZipFile

# Data processing
import numpy as np
import pandas as pd

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# Logistic regression
from statsmodels.tools import tools
from statsmodels.api import Logit

from sklearn.metrics import roc_curve

In [ ]:
# Original weather data needed the weather condition to be decoded
# Done using https://gist.github.com/kfish610/29d0e2874824732ec2d2922f979e4acb
# Due to slowness saved as a csv

# Further minimized using https://gist.github.com/kfish610/48605253428c856bbb1f461c10b6d6ac 

In [ ]:
# The original ASPM data (ASPM.zip) contains the hourly count of departures, as well as the percentage of delays
# We have to expand this into individual flights for the logistic regression
# Because we don't have the original flight data, we only have precision down to the hour
# Done using https://gist.github.com/kfish610/b63bbf488d91dcf6877925f7ddfe618b
# This also merges the weather data with the ASPM data, which in Python is too memory inefficient

zipFile = ZipFile('data/ASPM_Transformed.zip')

# Load, but also exclude COVID data
# This also excludes hail, as there are only 41 instances of hail in the whole dataset
df = pd.concat([
        pd.concat(pd.read_csv(
            zipFile.open(f"ASPM{year}.csv"), 
            chunksize=10000,
            usecols=range(1, 9)
        )) for year in range(2015, 2020)
    ])

# Lowercase for consistency
df.columns = df.columns.str.lower()

# Convert to category to save memory
df['facility'] = df['facility'].astype('category')

print(f"{df.shape}")
df.head()

In [ ]:
endog = df.delayed
exog = tools.add_constant(df.iloc[:, 2:], prepend=True)
full_model = Logit(endog, exog).fit(disp=False)
print(full_model.summary())

In [ ]:
## interaction (precipitation*visibility) and (snow*visibility)

In [ ]:
## everything except air_temp
endog = df.delayed
exog = tools.add_constant(df.drop(columns = ['air_temp']).iloc[:, 2:], prepend=True)
without_air_model = Logit(endog, exog).fit(disp=False)
print(without_air_model.summary())

In [ ]:
## thunder, snow, precipitation
endog = df.delayed
exog = tools.add_constant(df[['thunder', 'snow', 'precipitation']], prepend=True)
weather_model = Logit(endog, exog).fit(disp=False)
print(weather_model.summary())

In [ ]:
# sklearn's confusion matrix taxes 10x longer than this...
def confusion_matrix(y_true, y_pred):
    # Get all values that belong in a category
    def get_category(cat):
        return (y_true == cat[0]) & (y_pred == cat[1])
    
    # The four categories (first coordinate is true, second is predicted)
    categories = [(0, 0), (0, 1), (1, 0), (1, 1)]
    return np.apply_along_axis(get_category, arr=categories, axis=1).sum(axis=1).reshape(2, 2)

In [ ]:
# tpr also known as recall
# ppv also known as precision
def classification_scores(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    fpr = fp / (tn + fp)
    tpr = tp / (tp + fn)
    ppv = tp / (tp + fp)
    f1_score = 2 * (ppv * tpr) / (ppv + tpr)
    return (fpr, tpr, ppv, f1_score)

In [ ]:
thresholds = np.arange(0, 1, 0.05)
scores = np.empty((len(thresholds), 4))

for i, thresh in enumerate(thresholds):
    y_pred = np.where(full_model.predict() > thresh, 1, 0)
    scores[i] = classification_scores(df.delayed, y_pred)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
sns.lineplot(x=scores[:, 0], y=scores[:, 1])

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]), 
    np.max([ax.get_xlim(), ax.get_ylim()]), 
]

ax.plot(lims, lims, 'k:', alpha=0.5, zorder=0)
ax.set_aspect('equal', 'box')
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 8))
sns.countplot(y='facility', data=df, order=df.facility.value_counts().index)
ax.set_xlabel('Number of Flights (millions)')
ax.set_ylabel('Airport')
ax.set_title('Number of Flights by Airport (2015-2019)')

In [ ]:
delayRates = df.facility[df.delayed == 1].value_counts() / df.facility.value_counts()

fig, ax = plt.subplots(1, 1, figsize=(6, 8))
sns.barplot(y='facility', x='count', data=pd.DataFrame(delayRates).reset_index(), order=delayRates.sort_values(ascending=False).index)
ax.set_xlabel('Percent of Flights')
ax.set_ylabel('Airport')
ax.set_title('Percent Delayed by Airport (2015-2019)')

In [ ]:
pd.DataFrame(delayRates).reset_index()